In [1]:
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
from scipy.spatial.distance import cdist
from datetime import date
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

pd.options.mode.chained_assignment = None  # default='warn'

## Scraping and Cleaner Functions for Each Statistical Category

In [2]:
def rushing_scraper(url):
    # Open URL and pass to BeautifulSoup
    html = urlopen(url)
    stats_page = BeautifulSoup(html)
    # Collect table headers
    column_headers = stats_page.findAll('tr')[1]
    column_headers = [i.getText() for i in column_headers.findAll('th')]
    # Collect table rows
    rows = stats_page.findAll('tr')[2:]
    # Get stats from each row
    stats = []
    for i in range(len(rows)):
      stats.append([col.getText() for col in rows[i].findAll('td')])
    # Create DataFrame from our scraped data
    df = pd.DataFrame(stats, columns=column_headers[1:])
    return df

In [3]:
def receiving_scraper(url):
    # Open URL and pass to BeautifulSoup
    html = urlopen(url)
    stats_page = BeautifulSoup(html)
    # Collect table headers
    column_headers = stats_page.findAll('tr')[0]
    column_headers = [i.getText() for i in column_headers.findAll('th')]
    # Collect table rows
    rows = stats_page.findAll('tr')[1:]
    # Get stats from each row
    stats = []
    for i in range(len(rows)):
      stats.append([col.getText() for col in rows[i].findAll('td')])
    # Create DataFrame from our scraped data
    df = pd.DataFrame(stats, columns=column_headers[1:])
    return df

In [4]:
def passing_scraper(url):
    # Open URL and pass to BeautifulSoup
    html = urlopen(url)
    stats_page = BeautifulSoup(html)
    # Collect table headers
    column_headers = stats_page.findAll('tr')[0]
    column_headers = [i.getText() for i in column_headers.findAll('th')]
    column_headers[-6] = 'Sk_Yds'
    # Collect table rows
    rows = stats_page.findAll('tr')[1:]
    # Get stats from each row
    qb_stats = []
    for i in range(len(rows)):
      qb_stats.append([col.getText() for col in rows[i].findAll('td')])
    # Create DataFrame from our scraped data
    df = pd.DataFrame(qb_stats, columns=column_headers[1:])
    return df

In [5]:
def rushing_cleaner(data):
    #Filter for only QB, WR, RB and TE
    rush = data.loc[(data.Pos == 'QB') | (data.Pos == 'WR') | (data.Pos == 'RB') | (data.Pos == 'TE')]
    rush = rush.dropna()
    #Select Relevant stats and convert to numeric datatype
    rush = rush[['Player', 'Age','Pos', 'Tm', 'G', 'Att', 'Yds', 'TD', '1D', 'Fmb']]
    num_df = rush[['Age', 'G', 'Att', 'Yds', 'TD', '1D','Fmb']].astype(str).astype(float)
    #Identify Non-Numerical Columns and Clean the Player Name
    bio = rush[['Player', 'Pos', 'Tm']]
    rush = bio.merge(num_df, left_index=True, right_index=True)
    clean_name = []
    for name in bio.Player:
        clean = re.sub(r"[*+]+$", "", name)
        clean_name.append(clean)
    rush['Player'] = clean_name
    rush.rename(columns={'Att': 'Rush_Att', 'Yds': 'Rush_Yds', 'TD': 'Rush_TD', '1D': 'Rush_1D'}, inplace=True)
    #Group by Player, Position and Age to aggregate potential players traded mid-season
    rush = rush.groupby(by = ['Player', 'Pos', 'Age'], as_index = False).sum(numeric_only=True)
    rush.sort_values(by = 'Rush_Yds', ascending = False, inplace=True)
    #Drop duplicate players with the same names, and only keep the one with the more points
    rush.drop_duplicates(subset = ['Player'], keep = 'first', inplace = True)
    #Scale Data
    data = rush[['Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rush_1D']]
    scaler = MinMaxScaler()
    scaler.fit(data)
    scaled = np.round(scaler.transform(data), decimals = 3)
    rush[['Rush_Att_Scaled', 'Rush_Yds_Scaled', 'Rush_TD_Scaled', 'Rush_1D_Scaled']] = scaled
    return rush

In [6]:
def receiving_cleaner(data):
    #Filter Stats
    catch = data.loc[(data.Pos == 'WR') | (data.Pos == 'RB') | (data.Pos == 'TE')]
    catch = catch.loc[catch.Rec != '0']
    catch = catch.loc[catch.Tgt != '']
    catch = catch.loc[catch.Player != 'None']
    catch = catch.dropna()
    # Select Relevant stats and convert to numeric datatype
    catch = catch[['Player', 'Age', 'Tm', 'Pos', 'G', 'Tgt', 'Rec', 'Yds', '1D', 'TD']]
    num_df = catch[['Age', 'G', 'Tgt', 'Rec', 'Yds','1D', 'TD']].astype(str).astype(float)
    bio = catch[['Player', 'Pos', 'Tm']]
    #Identify Non-Numerical Columns and Clean the Player Name
    catch = bio.merge(num_df, left_index=True, right_index=True)
    clean_name = []
    for name in bio.Player:
        clean = re.sub(r"[*+]+$", "", name)
        clean_name.append(clean)
    catch['Player'] = clean_name
    catch.rename(columns={'Yds': 'Rec_Yds', 'TD': 'Rec_TD', '1D': 'Rec_1D'}, inplace=True)
    #Group by Player, Position and Age to aggregate potential players traded mid-season
    catch = catch.groupby(by = ['Player', 'Pos', 'Age'], as_index = False).sum(numeric_only=True)
    #Drop duplicate players with the same names, and only keep the one with the more points
    catch.sort_values(by = 'Rec_Yds', ascending = False, inplace=True)
    catch.drop_duplicates(subset = ['Player'], keep = 'first', inplace = True)
    #Scale Data
    data = catch[['Tgt','Rec', 'Rec_Yds','Rec_1D', 'Rec_TD']]
    scaler = MinMaxScaler()
    scaler.fit(data)
    scaled = np.round(scaler.transform(data),3)
    catch[['Tgt_Scaled','Rec_Scaled', 'Rec_Yds_Scaled','Rec_1D_Scaled', 'Rec_TD_Scaled']] = scaled
    return catch

In [7]:
def passing_cleaner(data):
    #Data Manipulation
    qb = data.loc[data.Pos == 'QB']
    qb = qb.dropna()
    # Select Relevant stats and convert to numeric datatype
    qb = qb[['Player', 'Age', 'Pos', 'G', 'Cmp', 'Att', 'Yds', '1D', 'TD', 'Int']]
    num_df = qb[['Age','G', 'Cmp', 'Att', 'Yds', '1D','TD', 'Int']].astype(str).astype(float)
    bio = qb[['Player', 'Pos']]
    qb = bio.merge(num_df, left_index=True, right_index=True)
    #Clean Player Name
    clean_name = []
    for name in qb.Player.unique():
        clean = re.sub(r"[*+]+$", "", name)
        clean_name.append(clean)
    qb['Player'] = clean_name
    qb.rename(columns={'Att': 'Pass_Att', 'Yds': 'Pass_Yds', '1D':'Pass_1D', 'TD': 'Pass_TD'}, inplace=True)
    #Group by Player, Position and Age to aggregate potential players traded mid-season
    qb = qb.groupby(by = ['Player', 'Pos', 'Age'], as_index = False).sum(numeric_only=True)
    data = qb[['Pass_Att', 'Pass_Yds','Pass_1D', 'Pass_TD']]
    scaler = MinMaxScaler()
    scaler.fit(data)
    scaled = np.round(scaler.transform(data),decimals = 3)
    qb[['Pass_Att_Scaled', 'Pass_Yds_Scaled','Pass_1D_Scaled', 'Pass_TD_Scaled']] = scaled
    return qb

## Organize Seasons

In [8]:
#Indicate Start and End Years for Season
start_dt = date(2022, 1, 1)
end_dt = date(2023, 1, 11)

year_range = [year for year in range(start_dt.year, end_dt.year +1)]

#For Loop to Iterate through Rushing, Passing and Receiving Seasons
categories = ['rushing', 'receiving', 'passing']

for i in year_range:
    for x in categories:
        url = 'https://www.pro-football-reference.com/years/{}/{}.htm'.format(i, x)
        scraper = x + '_scraper'
        cleaner = x + '_cleaner'
        data = vars()[scraper](url=url)
        df = vars()[cleaner](data)
        df['Season'] = i
        last_col = df.columns[-1]
        col_to_move = df.pop(last_col)
        df.insert(2, last_col, col_to_move)
        name = str(x)+'_df_' +str(i)
        vars()[name] = df
        time.sleep(1)
    print('Compiled stats from the {} season'.format(i))
    time.sleep(2)

Compiled stats from the 2022 season
Compiled stats from the 2023 season


In [191]:
#Concat Data
start_dt = date(2000, 1, 1)
end_dt = date(2022, 1, 11)

year_range = [year for year in range(start_dt.year, end_dt.year +1)]

x = min(year_range)
range_mod = year_range[1:]
for i in categories:
    df_name = str(i)+'_df_'+str(x)
    base_df = vars()[df_name]
    for y in range_mod:
        df_name = str(i)+'_df_' +str(y)
        year_df = vars()[df_name]
        base_df = pd.concat([base_df, year_df], ignore_index=True)
        vars()[str(i)+'_df'] = base_df
        
#Join the 3 categories 
df = passing_df.merge(rushing_df, how = 'outer', on = ['Player', 'Pos', 'Age', 'Season', 'G'])
df = df.merge(receiving_df, how = 'outer', on = ['Player', 'Pos', 'Age', 'Season', 'G'])
df.fillna(0, inplace=True)
#Calculate Fantasy Points
df['Fantasy_Points'] = round(df['Pass_Yds']*.04 + df['Pass_TD']*4+df['Int']*(-2)+df['Rush_Yds']*.1+df['Rush_TD']*6+df['Rec']*.5+df['Rec_Yds']*.1+df['Rec_TD']*6+df['Fmb']*(-2),0)
last_col = df.columns[-1]
col_to_move = df.pop(last_col)
df.insert(3, last_col, col_to_move)
df.sort_values(by = 'Fantasy_Points', ascending = False, inplace = True) #Sort DF by Fantasy Points
#df.drop_duplicates(subset = ['Player', 'Season'], keep = 'first', inplace = True)
df['Pos_Rank'] = df.groupby(['Pos', 'Season'])['Fantasy_Points'].rank(ascending = False, method = 'min')
df.loc[df.Pos != 'QB'].head()

,Player,Pos,Season,Fantasy_Points,Age,G,Cmp,Pass_Att,Pass_Yds,Pass_1D,...,Rec,Rec_Yds,Rec_1D,Rec_TD,Tgt_Scaled,Rec_Scaled,Rec_Yds_Scaled,Rec_1D_Scaled,Rec_TD_Scaled,Pos_Rank
2860,LaDainian Tomlinson,RB,2006,442.0,27.0,16.0,0.0,0.0,0.0,0.0,...,56.0,508.0,23.0,3.0,0.444,0.539,0.371,0.295,0.231,1.0
1700,Marshall Faulk,RB,2000,415.0,27.0,14.0,0.0,0.0,0.0,0.0,...,81.0,830.0,42.0,8.0,0.651,0.792,0.508,0.545,0.533,1.0
5739,Christian McCaffrey,RB,2019,409.0,23.0,16.0,0.0,0.0,0.0,0.0,...,116.0,1005.0,58.0,4.0,0.766,0.777,0.584,0.637,0.364,1.0
2259,Priest Holmes,RB,2003,408.0,30.0,16.0,0.0,0.0,0.0,0.0,...,74.0,690.0,35.0,0.0,0.489,0.629,0.408,0.417,0.000,1.0
2060,Priest Holmes,RB,2002,406.0,29.0,14.0,0.0,0.0,0.0,0.0,...,70.0,672.0,31.0,3.0,0.392,0.486,0.391,0.337,0.231,1.0


In [149]:
#Find duplicated players
duplicate_players = []
player_age_dup = df.Player[df.duplicated(subset = ['Player', 'Age'])].unique()
player_season_dup = df.Player[df.duplicated(subset = ['Player', 'Season'])].unique()
player_dup = np.union1d(player_age_dup, player_season_dup).tolist()
duplicated_df = df[df.Player.isin(player_dup)].sort_values(by = ['Player', 'Season', 'Age'], ascending = True)
duplicated_df = duplicated_df[['Player', 'Pos', 'Season', 'Age', 'Fantasy_Points', 'Pos_Rank']]
duplicated_df.head()

,Player,Pos,Season,Age,Fantasy_Points,Pos_Rank
2134,Adrian Peterson,RB,2002,23.0,19.0,73.0
2341,Adrian Peterson,RB,2003,24.0,8.0,94.0
2588,Adrian Peterson,RB,2004,25.0,6.0,101.0
2703,Adrian Peterson,RB,2005,26.0,57.0,55.0
2955,Adrian Peterson,RB,2006,27.0,24.0,75.0


- Adrian Peterson: 
    - drop duplicates for player and season (keep highest player value)
    - drop rows from before 2007
- Alex Smith: drop TE rows
- Antonio Brown: drop seasons before 2010
- Chris Henry: drop RB rows
- David Johnson: drop TE rows
- Justin Watson: drop all 
- Kevin Smith: drop WR
- Matt Jones: drop all
- Mike Thomas: drop everything before 2016
- Mike Williams: drop everything before 2017
- Reggie Brown: drop RB
- Ryan Grant: drop WR
- Ryan Griffin: drop all
- Zach Miller: drop all

In [193]:
#Remove Players altogether
df = df.loc[(df.Player != 'Zach Miller') & (df.Player != 'Ryan Griffin') & (df.Player != 'Justin Watson')
      & (df.Player != 'Matt Jones')]
#Individual Player Stats
alex = (df['Player'] == 'Alex Smith') & (df['Pos'] != 'TE')
ab = (df['Player'] == 'Antonio Brown') & (df['Season'] < 2010)
chris = (df['Player'] == 'Chris Henry') & (df['Pos'] != 'RB')
dj = (df['Player'] == 'David Johnson') & (df['Pos'] != 'TE')
kev = (df['Player'] == 'Kevin Smith') & (df['Pos'] != 'WR')
mt = (df['Player'] == 'Mike Thomas') & (df['Season'] < 2016)
mw = (df['Player'] == 'Mike Williams') & (df['Season'] < 2017)
reg = (df['Player'] == 'Reggie Brown') & (df['Pos'] != 'RB')
rg = (df['Player'] == 'Ryan Grant') & (df['Pos'] != 'WR')
df = df.drop(df[alex].index)
df = df.drop(df[ab].index)
df = df.drop(df[chris].index)
df = df.drop(df[dj].index)
df = df.drop(df[kev].index)
df = df.drop(df[mt].index)
df = df.drop(df[mw].index)
df = df.drop(df[reg].index)
df = df.drop(df[rg].index)

/var/folders/g9/lxmtwlzn7rb5ds2ctlg85t480000gn/T/ipykernel_86001/138042102.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.drop(df[ab].index)
/var/folders/g9/lxmtwlzn7rb5ds2ctlg85t480000gn/T/ipykernel_86001/138042102.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.drop(df[chris].index)
/var/folders/g9/lxmtwlzn7rb5ds2ctlg85t480000gn/T/ipykernel_86001/138042102.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.drop(df[dj].index)
/var/folders/g9/lxmtwlzn7rb5ds2ctlg85t480000gn/T/ipykernel_86001/138042102.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.drop(df[kev].index)
/var/folders/g9/lxmtwlzn7rb5ds2ctlg85t480000gn/T/ipykernel_86001/138042102.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.drop(df[mt].index)
/var/folders/g9/lxmtwlzn7rb5ds2ctlg85t480000gn/T/ipykernel_86

11421

In [194]:
ap = (df['Player'] == 'Adrian Peterson') & (df['Season'] < 2007)
ap2 = (df['Player'] == 'Adrian Peterson') & (df['Season'] == 2007)  & (df['Age'] == 28.0)
df = df.drop(df[ap].index)
df = df.drop(df[ap2].index)

len(df)

/var/folders/g9/lxmtwlzn7rb5ds2ctlg85t480000gn/T/ipykernel_86001/379094142.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.drop(df[ap2].index)


11415

In [195]:
df.to_csv('/Users/yushunli/Documents/Data Science/nfl_player_similarity/Datasets/season2000_to_2022_data.csv', index=False)

## Draft Classes

In [169]:
def draft_class_scraper(season):
    url = 'https://www.pro-football-reference.com/years/{}/draft.htm'.format(season)
    html = urlopen(url)
    stats_page = BeautifulSoup(html)
    # Collect table headers
    column_headers = stats_page.findAll('tr')[1]
    column_headers = [i.getText() for i in column_headers.findAll('th')]
    # Collect table rows
    rows = stats_page.findAll('tr')[2:]
    # Get stats from each row
    stats = []
    for i in range(len(rows)):
      stats.append([col.getText() for col in rows[i].findAll('td')])
    # Create DataFrame from our scraped data
    df = pd.DataFrame(stats, columns=column_headers[1:])
    #Filter for only relevant rows and columns
    df.dropna(inplace=True)
    df = df.loc[df.Player != None]
    df = df.loc[df.Player != 'None']
    draft = df[['Player', 'Pos', 'Age', 'Pick']]
    #draft['Age','Pick'] = draft[['Age','Pick']].astype(str).astype(float)
    draft = draft.loc[draft.Pick != None]
    draft.reset_index(drop = True, inplace=True)
    draft.loc[:, 'Season'] = season
    draft['Pick'] = draft.index + 1
    draft = draft.loc[(draft.Pos == 'QB') | (draft.Pos == 'WR') | (draft.Pos == 'RB') | (draft.Pos == 'TE')]
    return draft

In [170]:
#Indicate Start and End Years for Season
start_dt = date(1994, 1, 1)
end_dt = date(2022, 1, 11)

year_range = [year for year in range(start_dt.year, end_dt.year +1)]

for i in year_range:
    df = draft_class_scraper(season = i)
    name = 'draftclass_' +str(i)
    vars()[name] = df
    print('Completed the {} Draft Class'.format(i))
    time.sleep(2)

Completed the 1994 Draft Class
Completed the 1995 Draft Class
Completed the 1996 Draft Class
Completed the 1997 Draft Class
Completed the 1998 Draft Class
Completed the 1999 Draft Class
Completed the 2000 Draft Class
Completed the 2001 Draft Class
Completed the 2002 Draft Class
Completed the 2003 Draft Class
Completed the 2004 Draft Class
Completed the 2005 Draft Class
Completed the 2006 Draft Class
Completed the 2007 Draft Class
Completed the 2008 Draft Class
Completed the 2009 Draft Class
Completed the 2010 Draft Class
Completed the 2011 Draft Class
Completed the 2012 Draft Class
Completed the 2013 Draft Class
Completed the 2014 Draft Class
Completed the 2015 Draft Class
Completed the 2016 Draft Class
Completed the 2017 Draft Class
Completed the 2018 Draft Class
Completed the 2019 Draft Class
Completed the 2020 Draft Class
Completed the 2021 Draft Class
Completed the 2022 Draft Class


In [179]:
for i in year_range:
    name = 'draftclass_' +str(i)
    draft_class = pd.concat([draft_class,vars()[name]])
    
clean_name = []
for name in draft_class.Player:
    clean = re.sub(r" HOF$", "", name)
    clean_name.append(clean)

draft_class['Player'] = clean_name
draft_class.drop_duplicates(subset = ['Player', 'Age', 'Pick', 'Season'], keep = 'first', inplace = True)
draft_class["Position_Pick"] = draft_class.groupby(['Pos', 'Season']).cumcount()+1
draft_class = draft_class.loc[(draft_class.Pos == 'QB') | (draft_class.Pos == 'WR') | (draft_class.Pos == 'RB') | (draft_class.Pos == 'TE')]
draft_class.to_csv('/Users/yushunli/Documents/Data Science/nfl_player_similarity/Datasets/1994_to_2022_draftclass.csv',index=False)
draft_class

,Player,Pos,Age,Pick,Season,Position_Pick
1,Marshall Faulk,RB,21,2,1994,1
2,Heath Shuler,QB,22,3,1994,1
5,Trent Dilfer,QB,22,6,1994,2
16,Charles Johnson,WR,22,17,1994,1
20,Johnnie Morton,WR,22,21,1994,2
...,...,...,...,...,...,...
250,Isiah Pacheco,RB,22,251,2022,22
257,Samori Toure,WR,24,258,2022,28
259,Zander Horvath,RB,23,260,2022,23
261,Brock Purdy,QB,22,262,2022,9


In [181]:
draft_class.loc[draft_class.Player == 'Josh Gordon']

,Player,Pos,Age,Pick,Season,Position_Pick
253,Josh Gordon,WR,21,254,2012,34


In [189]:
df.head()

,Player,Pos,Age,Pick,Season
7,Drake London,WR,21,8,2022
9,Garrett Wilson,WR,22,10,2022
10,Chris Olave,WR,22,11,2022
11,Jameson Williams,WR,21,12,2022
15,Jahan Dotson,WR,22,16,2022


In [199]:
first_season = df.groupby(by = ['Player', 'Pos'], as_index = False).min('Age')
draft_df = first_season.merge(draft_class, on = ['Player', 'Pos', 'Season'])
#draft_df = draft_df[['Player', 'Pos', 'Pick', 'Position Pick Number', 'Season_x']]
#draft_df.rename(columns = {'Season_y' : 'Draft_Year', 'Position Pick Number': 'Position_Pick'}, inplace=True)
#draft_df.sort_values(by = ['Draft_Year', 'Pick'], ascending = True, inplace=True)
#draft_df.reset_index(drop=True, inplace=True)
draft_df

,Player,Pos,Season,Fantasy_Points,Age_x,G,Cmp,Pass_Att,Pass_Yds,Pass_1D,...,Rec_TD,Tgt_Scaled,Rec_Scaled,Rec_Yds_Scaled,Rec_1D_Scaled,Rec_TD_Scaled,Pos_Rank,Age_y,Pick,Position_Pick
0,A.J. Brown,WR,2019,149.0,22.0,13.0,0.0,0.0,0.0,0.0,...,5.0,0.451,0.345,0.447,0.429,0.312,5.0,22,51,4
1,A.J. Feeley,QB,2001,12.0,24.0,1.0,10.0,14.0,143.0,8.0,...,0.0,0.000,0.000,0.000,0.000,0.000,30.0,24,155,9
2,A.J. Green,WR,2011,48.0,23.0,9.0,0.0,0.0,0.0,0.0,...,2.0,0.251,0.181,0.131,0.138,0.111,4.0,23,4,1
3,AJ Dillon,RB,2020,39.0,22.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.006,0.008,0.016,0.013,0.000,23.0,22,62,6
4,Aaron Brown,RB,2009,1.0,24.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.007,0.000,0.000,78.0,23,192,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,Zach Wilson,QB,2021,92.0,22.0,9.0,132.0,242.0,1688.0,73.0,...,0.0,0.000,0.000,0.000,0.000,0.000,30.0,22,2,2
1249,Zack Moss,RB,2020,59.0,23.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.066,0.079,0.023,0.025,0.000,44.0,22,86,9
1250,Zamir White,RB,2022,7.0,23.0,14.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.000,0.000,0.000,115.0,22,122,8
1251,Zay Jones,WR,2017,28.0,22.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.115,0.103,0.102,0.076,0.000,26.0,22,37,4


In [212]:
players = draft_df.Player.unique()
not_in = df[~df.Player.isin(players)]
not_in_df = not_in.groupby(by = ['Player', 'Pos'], as_index = False).min('Age')
not_in_df.sort_values(by = 'Fantasy_Points', ascending = False).head(10)

,Player,Pos,Season,Fantasy_Points,Age,G,Cmp,Pass_Att,Pass_Yds,Pass_1D,...,Rec,Rec_Yds,Rec_1D,Rec_TD,Tgt_Scaled,Rec_Scaled,Rec_Yds_Scaled,Rec_1D_Scaled,Rec_TD_Scaled,Pos_Rank
1398,Robert Smith,RB,2000,257.0,28.0,16.0,0.0,0.0,0.0,0.0,...,36.0,348.0,15.0,3.0,0.250,0.347,0.214,0.195,0.200,10.0
1558,Tiki Barber,RB,2000,188.0,25.0,14.0,0.0,0.0,0.0,0.0,...,52.0,461.0,20.0,0.0,0.406,0.479,0.274,0.261,0.000,1.0
1604,Travis Etienne,RB,2022,182.0,23.0,17.0,0.0,0.0,0.0,0.0,...,35.0,316.0,16.0,0.0,0.240,0.268,0.176,0.200,0.000,18.0
1607,Travis Kelce,TE,2014,150.0,25.0,15.0,0.0,0.0,0.0,0.0,...,67.0,862.0,40.0,4.0,0.470,0.516,0.469,0.430,0.286,1.0
807,Jeff Graham,WR,2000,137.0,31.0,14.0,0.0,0.0,0.0,0.0,...,52.0,811.0,38.0,4.0,0.609,0.455,0.508,0.494,0.267,29.0
1588,Tony Gonzalez,TE,2000,137.0,24.0,15.0,0.0,0.0,0.0,0.0,...,63.0,656.0,39.0,2.0,0.480,0.437,0.450,0.424,0.167,1.0
1539,Terrell Owens,WR,2000,136.0,27.0,7.0,0.0,0.0,0.0,0.0,...,47.0,763.0,32.0,5.0,0.535,0.443,0.488,0.432,0.385,2.0
579,Elvis Grbac,QB,2000,135.0,30.0,14.0,265.0,467.0,3033.0,151.0,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,6.0
1670,Warrick Dunn,RB,2000,134.0,25.0,11.0,0.0,0.0,0.0,0.0,...,22.0,170.0,5.0,0.0,0.152,0.206,0.124,0.068,0.000,12.0
852,Jimmy Smith,WR,2000,132.0,31.0,12.0,0.0,0.0,0.0,0.0,...,54.0,805.0,38.0,4.0,0.610,0.457,0.476,0.452,0.235,5.0


In [213]:
not_in_df = not_in_df.loc[not_in_df.Player != 'Travis Kelce']
not_in_df = not_in_df.loc[not_in_df.Player != 'Travis Etienne']
not_in_df = not_in_df.loc[not_in_df.Season < 2020]

In [239]:
not_in_df[['Player', 'Fantasy_Points', 'Pos']].groupby(by = ['Player', 'Pos'], as_index = False).sum().sort_values(by = 'Fantasy_Points', ascending = False).head(25)

,Player,Pos,Fantasy_Points
1259,Robert Smith,RB,257.0
1405,Tiki Barber,RB,188.0
1432,Tony Gonzalez,TE,137.0
724,Jeff Graham,WR,137.0
1387,Terrell Owens,WR,136.0
526,Elvis Grbac,QB,135.0
1500,Warrick Dunn,RB,134.0
768,Jimmy Smith,WR,132.0
410,David Nelson,TE,126.0
331,Curtis Martin,RB,123.0


In [216]:
players2 = not_in_df.Player.unique()
df_edited = df[~df.Player.isin(players2)]
len(df_edited)

6430

In [236]:
total_points = df_edited[['Player', 'Fantasy_Points', 'Pos']].groupby(by = ['Player', 'Pos'], as_index = False).sum().sort_values(by = 'Fantasy_Points', ascending = False)
season_played = df_edited[['Player', 'Season', 'Pos']].groupby(by = ['Player', 'Pos'], as_index = False).count().sort_values(by = 'Season', ascending = False)
agg_df = total_points.merge(season_played, how = 'left', on = ['Player', 'Pos'])
agg_df['Points_Per_Season'] = round(agg_df['Fantasy_Points'] / agg_df['Season'],0)
agg_df.loc[agg_df.Pos != 'QB'].sort_values(by = 'Points_Per_Season', ascending = False).head(25)

,Player,Pos,Fantasy_Points,Season,Points_Per_Season
10,LaDainian Tomlinson,RB,3068.0,11,279.0
215,Justin Jefferson,WR,800.0,3,267.0
71,Alvin Kamara,RB,1508.0,6,251.0
247,Jonathan Taylor,RB,716.0,3,239.0
84,Christian McCaffrey,RB,1411.0,6,235.0
57,Ezekiel Elliott,RB,1648.0,7,235.0
89,Todd Gurley,RB,1398.0,6,233.0
373,Najee Harris,RB,465.0,2,232.0
377,Ja'Marr Chase,WR,461.0,2,230.0
61,Tyreek Hill,WR,1607.0,7,230.0


In [ ]:
df_edited.to_csv('', index = False)